<h1>ECOSISTEMA SPARK</h1>
<h1>Práctica 2: SparkSQL</h1>
<h1>TAREA 1</h1>


<h4>Autor: Guillermo García López</h4>

In [1]:
import time as t
init_time = t.time()

In [2]:
# Comando para habilitar scrolling horizontal en celdas
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
# Se crea sparksession para utilidades SparkSQL
from pyspark.sql import SparkSession, Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL - Practica 2") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
# Se crea SparkContext() para utilidades sobre RDD's
sc = SparkContext.getOrCreate()

Obtenemos RDD para cada fichero:

In [5]:
'''
Función para crear un RDD a partir del fichero .csv original, con el formato de la utilidad "Row" de SparkSQL 
en base al esquema inferido del fichero:
[0] -> Index           (int)
[1] -> Arrival_Time    (int)
[2] -> Creation_Time   (int)
[3] -> x               (float)
[4] -> y               (float)
[5] -> z               (float)
[6] -> User            (string)
[7] -> Model           (string)
[8] -> Device          (string)
[9] -> Gt              (string)

Input: nombre de fichero a importar.
Output: RDD con el esquema de campos en formato "Row"
'''
def create_RDD(file):
    rdd = sc.textFile(file) \
            .map(lambda el: el.split(',')) \
            .map(lambda el: Row( user=str(el[6]), model=str(el[7]), gt=str(el[9]), x=float(el[3]), \
                                 y=float(el[4]), z=float(el[5]) ) )
            
            #.map(lambda el: Row(index=int(el[0]), arrTime=int(el[1]), crTime=int(el[2]), x=float(el[3]), \
            #                     y=float(el[4]), z=float(el[5]), user=str(el[6]), model=str(el[7]), \
            #                     device=str(el[8]), gt=str(el[9])))
            
    return rdd

In [6]:
'''
Función para crear un dataframe y registrarlo como tabla a partir de un RDD. Los nombres de las tablas 
serán los siguientes:
Phones_accelerometer.csv -> phonesAcc
Phones_gyroscope.csv     -> phonesGyr
Watch_accelerometer.csv  -> watchesAcc
Watch_gyroscope.csv      -> watchesGyr

Input: RDD
Output: none
'''
def create_reg_df(rdd):
    df = spark.createDataFrame(rdd)
    return df

Se crean los RDD's para cada fichero:

In [7]:
#phAccRDD  = create_RDD('./large_data/datos_sensores/activity_recognition_exp/Phones_accelerometer.csv')
#phGyrRDD  = create_RDD('./large_data/datos_sensores/activity_recognition_exp/Phones_gyroscope.csv')
#watAccRDD = create_RDD('./large_data/datos_sensores/activity_recognition_exp/Watch_accelerometer.csv')
#watGyrRDD = create_RDD('./large_data/datos_sensores/activity_recognition_exp/Watch_gyroscope.csv')
phAccRDD  = create_RDD('./small_data/Phones_accelerometer.csv')
phGyrRDD  = create_RDD('./small_data/Phones_gyroscope.csv')
watAccRDD = create_RDD('./small_data/Watch_accelerometer.csv')
watGyrRDD = create_RDD('./small_data/Watch_gyroscope.csv')

A partir de los RDD's se crean los dataframes:

In [8]:
dfPhAcc  = spark.createDataFrame(phAccRDD)
dfPhGyr  = spark.createDataFrame(phGyrRDD)
dfWatAcc = spark.createDataFrame(watAccRDD)
dfWatGyr = spark.createDataFrame(watGyrRDD)

In [9]:
#dfPhAcc  = create_reg_df(phAccRDD)
#dfPhGyr  = create_reg_df(phGyrRDD)
#dfWatAcc = create_reg_df(watAccRDD)
#dfWatGyr = create_reg_df(watGyrRDD)

In [10]:
phonesStats = dfPhAcc.join(dfPhGyr, ['user', 'model', 'gt', 'x', 'y', 'z']  ,'full')

In [11]:
watchesStats = dfWatAcc.join(dfWatGyr, ['user', 'model', 'gt', 'x', 'y', 'z']  ,'full')

Se unen los dataframes mediante 'union', registrando una tabla del dataframe resultante:

In [12]:
Stats = phonesStats.unionAll(watchesStats)

In [13]:
Stats.createOrReplaceTempView('statsTable')

Y se ejecuta la query que retorna los resultados:

In [14]:
query = "SELECT user, model ,gt ,mean(x), mean(y), mean(z) \
                ,std(x), std(y), std(z), max(x), max(y), max(z), min(x) \
                ,min(y), min(z) \
         FROM statsTable \
         GROUP BY user, model, gt"

In [15]:
result = spark.sql(query)
#result.show()

In [16]:
end_time = t.time()
def foo():
    interval_time = end_time - init_time
    print(interval_time)
foo()

9.868576765060425


In [17]:
result.show()

+----+------+-----+-------------------+-------------------+-------------------+-----------------+-------------------+-------------------+------------------+------------------+------------+-----------+-----------+-------------------+
|user| model|   gt|             avg(x)|             avg(y)|             avg(z)|   stddev_samp(x)|     stddev_samp(y)|     stddev_samp(z)|            max(x)|            max(y)|      max(z)|     min(x)|     min(y)|             min(z)|
+----+------+-----+-------------------+-------------------+-------------------+-----------------+-------------------+-------------------+------------------+------------------+------------+-----------+-----------+-------------------+
|   a|  gear| null| -4.625535971408151|-1.7571181213697233|-0.5569131464861713|4.644796091579758| 1.7196641004044204|0.48540930263564325|         0.0561927|      -0.018109497|-0.055127434| -9.3583355| -3.5942953|         -1.1420342|
|   a|nexus4|stand|-3.0124555493104537|0.46725270568362365|  4.00704